# HW3 Image Classification
## We strongly recommend that you run with Kaggle for this homework


# Get Data
Notes: if the links are dead, you can download the data directly from Kaggle and upload it to the workspace, or you can use the Kaggle API to directly download the data into colab.


In [2]:
# ! wget https://www.dropbox.com/s/6l2vcvxl54b0b6w/food11.zip

In [3]:
# ! unzip food11.zip

# Training

In [4]:
_exp_name = "sample"

In [5]:
# Import necessary packages.
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset

# This is for the progress bar.
from tqdm.auto import tqdm
import random

In [6]:
myseed = 2222  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

## **Transforms**
Torchvision provides lots of useful utilities for image preprocessing, data wrapping as well as data augmentation.

Please refer to PyTorch official website for details about different transforms.

In [7]:
# Normally, We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# However, it is also possible to use augmentation in the testing phase.
# You may use train_tfm to produce a variety of images and then test using ensemble methods
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    transforms.RandomRotation(degrees=(0, 180)),
    transforms.RandomCrop(size=(128, 128)),
    transforms.RandomHorizontalFlip(p=0.5),
    # transforms.ColorJitter(brightness=.5, hue=.3),
    transforms.RandomAdjustSharpness(sharpness_factor=2),
    transforms.ToTensor(),
])


## **Datasets**
The data is labelled by the name, so we load images and label while calling '__getitem__'

In [8]:
class FoodDataset(Dataset):

    def __init__(self, path, tfm=test_tfm, files = None):
        super(FoodDataset).__init__()
        self.path = path
        self.files = sorted([os.path.join(path, x) for x in os.listdir(path) if x.endswith(".jpg")])
        if files != None:
            self.files = files
        print(f"One {path} sample",self.files[0])
        self.transform = tfm
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self, idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)
        #im = self.data[idx]
        try:
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1 # test has no label
        return im,label



In [9]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 維度 [3, 128, 128]
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),  # [64, 128, 128]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [64, 64, 64]

            nn.Conv2d(64, 128, 3, 1, 1), # [128, 64, 64]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 32, 32]

            nn.Conv2d(128, 256, 3, 1, 1), # [256, 32, 32]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [256, 16, 16]

            nn.Conv2d(256, 512, 3, 1, 1), # [512, 16, 16]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 8, 8]
            
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 8, 8]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 4, 4]
        )
        self.avgpool = nn.AdaptiveAvgPool2d((4, 4))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(512*4*4, 1024),
            nn.ReLU(),
            nn.Dropout(0.4), # added
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 11)
        )

    def forward(self, x):
        out = self.cnn(x)
        out = self.avgpool(out)
        out = out.view(out.size()[0], -1)
        return self.fc(out)

In [18]:
class Residual_Network(nn.Module):
    def __init__(self):
        super(Residual_Network, self).__init__()

        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)        
        self.cnn_layer1 = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
        )

        self.cnn_layer2 = nn.Sequential(
            nn.Conv2d(64, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
        )
        self.cnn_layer2_direct = nn.Conv2d(64, 64, 1, 1, 0)
             
        self.cnn_layer3 = nn.Sequential(
            nn.Conv2d(64, 128, 3, 2, 1),
            nn.BatchNorm2d(128),
        )

        self.cnn_layer4 = nn.Sequential(
            nn.Conv2d(128, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
        )
        self.cnn_layer4_direct = nn.Conv2d(128, 128, 1, 1, 0)
        
        self.cnn_layer5 = nn.Sequential(
            nn.Conv2d(128, 256, 3, 2, 1),
            nn.BatchNorm2d(256),
        )
        self.cnn_layer6 = nn.Sequential(
            nn.Conv2d(256, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
        )
        self.cnn_layer6_direct = nn.Conv2d(256, 256, 1, 1, 0)
        
        self.fc_layer = nn.Sequential(
            nn.Linear(256* 32* 32, 256),
            nn.ReLU(),
            nn.Linear(256, 11)
        )
        self.relu = nn.ReLU()

    def forward(self, x):
        # input (x): [batch_size, 3, 128, 128]
        # output: [batch_size, 11]

        # Extract features by convolutional layers.
        x1 = self.cnn_layer1(x) # [64, 128, 128]       
        x1 = self.relu(x1)

        
        x2 = self.cnn_layer2(x1) # [64, 128, 128]
        y2 = self.cnn_layer2_direct(x1)         
        x2 = self.relu(x2 + y2)

#         nn.Conv2d(64, 128, 3, 2, 1),       
        # not residual block
        x3 = self.cnn_layer3(x2) # [128, 64, 64]  
        x3 = self.relu(x3)
            
        # residual block
        # nn.Conv2d(128, 128, 3, 1, 1),
        x4 = self.cnn_layer4(x3) # [128, 64, 64]
        y4 = self.cnn_layer4_direct(x3)
        x4 = self.relu(x4 + y4)
        
         # not residual block
        # nn.Conv2d(128, 256, 3, 2, 1), 
        x5 = self.cnn_layer5(x4) #  [256, 32, 32]         
        x5 = self.relu(x5)
        
        # residual block
        # nn.Conv2d(256, 256, 3, 1, 1),
        x6 = self.cnn_layer6(x5) # [256, 32, 32]
        y6 = self.cnn_layer6_direct(x5)           
        x6 = self.relu(x6 + y6)
        
        # The extracted feature map must be flatten before going to fully-connected layers.
        xout = x6.flatten(1)

        # The features are transformed by fully-connected layers to obtain the final logits.
        xout = self.fc_layer(xout)
        return xout

In [19]:
batch_size = 64
_dataset_dir = "/kaggle/input/food11/food11"

# Construct datasets.
# The argument "loader" tells how torchvision reads the data.
train_set = FoodDataset(os.path.join(_dataset_dir,"training"), tfm=train_tfm)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

valid_set = FoodDataset(os.path.join(_dataset_dir,"validation"), tfm=test_tfm)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

print(len(train_set))
print(len(valid_set))

merged_train_set = torch.utils.data.ConcatDataset([train_set, valid_set])
print(len(merged_train_set))

One /kaggle/input/food11/food11/training sample /kaggle/input/food11/food11/training/0_0.jpg
One /kaggle/input/food11/food11/validation sample /kaggle/input/food11/food11/validation/0_0.jpg
9866
3430
13296


In [20]:
# "cuda" only when GPUs are available.
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cuda"
patience = 8 # If no improvement in 'patience' epochs, early stop
n_epochs = 80

from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights

def train_and_evaluate(epoch, model, train_loader, valid_loader, criterion, optimizer):
    model.train()

    # These are used to record information in training.
    train_loss = []
    train_accs = []

    for batch in tqdm(train_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        #imgs = imgs.half()
        #print(imgs.shape,labels.shape)

        # Forward the data. (Make sure data and model are on the same device.)
        logits = model(imgs.to(device))

        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
        loss = criterion(logits, labels.to(device))

        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        loss.backward()

        # Clip the gradient norms for stable training.
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        # Update the parameters with computed gradients.
        optimizer.step()

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        train_loss.append(loss.item())
        train_accs.append(acc)
        
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)

    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    model.eval()

    # These are used to record information in validation.
    valid_loss = []
    valid_accs = []

    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        #imgs = imgs.half()

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            logits = model(imgs.to(device))

        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels.to(device))

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        valid_loss.append(loss.item())
        valid_accs.append(acc)
        #break

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    # Print the information.
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

    return valid_acc, valid_loss

In [21]:
from sklearn.model_selection import ShuffleSplit
from torchvision import models

# model = Classifier().to(device)
model = Residual_Network().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0002, weight_decay=1e-5)

stale = 0
best_acc = 0

k_folds = 5
rs = ShuffleSplit(n_splits=k_folds, test_size=.1, random_state=1981)
epoch = 0

while epoch < n_epochs and stale <= patience:    
    for fold, (train_idx, valid_idx) in enumerate(rs.split(merged_train_set)):               
        train_subset = Subset(merged_train_set, train_idx)
        valid_subset = Subset(merged_train_set, valid_idx)
        
        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        valid_loader = DataLoader(valid_subset, batch_size=batch_size, shuffle=False)
        
        valid_acc, valid_loss = train_and_evaluate(epoch, model, train_loader, valid_loader, criterion, optimizer)
    
        # update logs
        if valid_acc > best_acc:
            with open(f"./{_exp_name}_log.txt","a"):
                print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best")
        else:
            with open(f"./{_exp_name}_log.txt","a"):
                print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")
        
        # save models
        if valid_acc > best_acc:
            print(f"Best model found at epoch {epoch}, saving model")
            torch.save(model.state_dict(), f"/kaggle/working/{_exp_name}_best.ckpt") # only save best to prevent output memory exceed error
            best_acc = valid_acc
            stale = 0
        else:
            stale += 1
            if stale > patience:
                print(f"No improvment {patience} consecutive epochs, early stopping")
                break
       
        epoch += 1

  0%|          | 0/187 [00:00<?, ?it/s]

[ Train | 001/080 ] loss = 2.97595, acc = 0.28088


  0%|          | 0/21 [00:00<?, ?it/s]

[ Valid | 001/080 ] loss = 1.84374, acc = 0.35432
[ Valid | 001/080 ] loss = 1.84374, acc = 0.35432 -> best
Best model found at epoch 0, saving model


  0%|          | 0/187 [00:00<?, ?it/s]

...

[ Valid | 079/080 ] loss = 0.16009, acc = 0.94634
[ Valid | 079/080 ] loss = 0.16009, acc = 0.94634


  0%|          | 0/187 [00:00<?, ?it/s]

[ Train | 080/080 ] loss = 0.10336, acc = 0.96490


  0%|          | 0/21 [00:00<?, ?it/s]

[ Valid | 080/080 ] loss = 0.13707, acc = 0.94878
[ Valid | 080/080 ] loss = 0.13707, acc = 0.94878


In [ ]:
test_set = FoodDataset(os.path.join(_dataset_dir,"test"), tfm=test_tfm)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

# Testing and generate prediction CSV

In [ ]:
model_best = Classifier().to(device)
model_best.load_state_dict(torch.load(f'/kaggle/working/{_exp_name}_best.ckpt'))
model_best.eval()
prediction = []

with torch.no_grad():
    for data,_ in test_loader:
        test_pred = model_best(data.to(device))
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
         
        prediction += test_label.squeeze().tolist()

In [ ]:
#create test csv
def pad4(i):
    return "0"*(4-len(str(i)))+str(i)
df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(1,len(test_set)+1)]
df["Category"] = prediction
df.to_csv("submission.csv",index = False)

# Q1. Augmentation Implementation
## Implement augmentation by finishing train_tfm in the code with image size of your choice. 
## Directly copy the following block and paste it on GradeScope after you finish the code
### Your train_tfm must be capable of producing 5+ different results when given an identical image multiple times.
### Your  train_tfm in the report can be different from train_tfm in your training code.


In [ ]:
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    transforms.RandomRotation(degrees=(0, 180)),
    transforms.RandomCrop(size=(128, 128)),
    transforms.RandomHorizontalFlip(p=0.5),
    # transforms.ColorJitter(brightness=.5, hue=.3),
    transforms.RandomAdjustSharpness(sharpness_factor=2),
    transforms.ToTensor(),
])

# Q2. Residual Implementation
![](https://i.imgur.com/GYsq1Ap.png)
## Directly copy the following block and paste it on GradeScope after you finish the code


In [12]:
from torch import nn
class Residual_Network(nn.Module):
    def __init__(self):
        super(Residual_Network, self).__init__()

        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)        
        self.cnn_layer1 = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
        )

        self.cnn_layer2 = nn.Sequential(
            nn.Conv2d(64, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
        )
        self.cnn_layer2_direct = nn.Conv2d(64, 64, 1, 1, 0)
             
        self.cnn_layer3 = nn.Sequential(
            nn.Conv2d(64, 128, 3, 2, 1),
            nn.BatchNorm2d(128),
        )

        self.cnn_layer4 = nn.Sequential(
            nn.Conv2d(128, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
        )
        self.cnn_layer4_direct = nn.Conv2d(128, 128, 1, 1, 0)
        
        self.cnn_layer5 = nn.Sequential(
            nn.Conv2d(128, 256, 3, 2, 1),
            nn.BatchNorm2d(256),
        )
        self.cnn_layer6 = nn.Sequential(
            nn.Conv2d(256, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
        )
        self.cnn_layer6_direct = nn.Conv2d(256, 256, 1, 1, 0)
        
        self.fc_layer = nn.Sequential(
            nn.Linear(256* 32* 32, 256),
            nn.ReLU(),
            nn.Linear(256, 11)
        )
        self.relu = nn.ReLU()

    def forward(self, x):
        # input (x): [batch_size, 3, 128, 128]
        # output: [batch_size, 11]

        # Extract features by convolutional layers.
        x1 = self.cnn_layer1(x) # [64, 128, 128]       
        x1 = self.relu(x1)
        
        x2 = self.cnn_layer2(x1) # [64, 128, 128]
        y2 = self.cnn_layer2_direct(x1)         
        x2 = self.relu(x2 + y2)
     
        # not residual block
        # nn.Conv2d(64, 128, 3, 2, 1),     
        x3 = self.cnn_layer3(x2) # [128, 64, 64]  
        x3 = self.relu(x3)
            
        # residual block
        # nn.Conv2d(128, 128, 3, 1, 1),
        x4 = self.cnn_layer4(x3) # [128, 64, 64]
        y4 = self.cnn_layer4_direct(x3)
        x4 = self.relu(x4 + y4)
        
        # not residual block
        # nn.Conv2d(128, 256, 3, 2, 1), 
        x5 = self.cnn_layer5(x4) #  [256, 32, 32]         
        x5 = self.relu(x5)
        
        # residual block
        # nn.Conv2d(256, 256, 3, 1, 1),
        x6 = self.cnn_layer6(x5) # [256, 32, 32]
        y6 = self.cnn_layer6_direct(x5)           
        x6 = self.relu(x6 + y6)
        
        # The extracted feature map must be flatten before going to fully-connected layers.
        xout = x6.flatten(1)

        # The features are transformed by fully-connected layers to obtain the final logits.
        xout = self.fc_layer(xout)
        return xout